In [1]:
import numpy as np
import torch
import os
os.chdir("../")

from torch.utils.data import DataLoader
from data.torch_dataset import LetterVectors
from data.transform import ToFixedTensor
from config.config_loader import load_config
from model.net import CNN
from utils.metric import Metrics
from shapely.geometry import LineString

In [2]:
config = load_config()
device = 'cuda' if torch.cuda.is_available() else 'cpu'